In [8]:
import cPickle
import os
import pandas as pd
import multiprocessing
from multiprocessing.pool import Pool
import boto3

In [2]:
zappos = pd.read_csv('zap.csv')

In [3]:
zappos['brand_name_clean'] = zappos.brand_name.apply(lambda x: x.strip().lower())
zappos['shoe_name_clean'] = zappos.shoe_name.apply(lambda x: x.strip().lower())

In [4]:
brands = zappos.brand_name_clean.unique()
shoes = zappos.shoe_name_clean.unique()


In [5]:
brands_dict = dict(zip(brands, range(len(brands))))
shoes_dict = dict(zip(shoes, range(len(shoes))))

In [20]:
def get_shoe_code(row):
    brand_code = brands_dict[row['brand_name_clean']]
    shoe_code = shoes_dict[row['shoe_name_clean']]
    color_code = row['colorcode']
    return str(brand_code) + '_' + str(shoe_code) + '_' + str(color_code)

def get_img_array(url, resize=None):
    img = np.array(bytearray(urllib.urlopen(url).read()), dtype='uint8')
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)
    try:
        img = cv2.resize(img, resize)
        return img
    except Exception, e:
        raise Exception(e)
        
def grab_images(row):
    view_points = [u'lateralfrontleft', u'heelview', u'toeview', u'medialfrontright', u'topview']
    row_imgs = map(lambda url: get_img_array(url, (224, 224)), row[1][view_points].tolist())
    shoe_views = dict(zip(view_points, row_imgs))
    fname = get_shoe_code(row[1])
    cPickle.dump(shoe_views, open(fname, 'wb'))
    print'Success, Downloading: {}'.format(fname)
    return fname 

        
def create_s3_objects(s3, row, Bucket='zappos_shoes'):
    try:
        fname = grab_images(row)
        s3.put_object(Bucket=Bucket, Key=fname, Body=open(fname, 'rb'))
        print 'Succes, Uploaded to S3: {}'.format(fname)
        os.remove(fname)
    except:
        fname = get_shoe_code(row[1])
        print 'Failure: {}'.format(fname)
        pass


In [21]:
s3_connection = boto3.client('s3')
def zappos_connection(x):
    return create_s3_objects(s3_connection, x)

In [31]:
list(zappos.iterrows())[0]

(0, brand_name                                                       Nike
 shoe_name                                                Flex RN 2017
 colorway                 Black/Metallic Hematite/Anthracite/Dark Grey
 colorcode                                                      684832
 lateralfrontleft    http://www.zappos.com/images/z/3/9/3/9/7/0/393...
 heelview            http://www.zappos.com/images/z/3/9/3/9/7/0/393...
 toeview             http://www.zappos.com/images/z/3/9/3/9/7/0/393...
 medialfrontright    http://www.zappos.com/images/z/3/9/3/9/7/0/393...
 topview             http://www.zappos.com/images/z/3/9/3/9/7/0/393...
 brand_name_clean                                                 nike
 shoe_name_clean                                          flex rn 2017
 Name: 0, dtype: object)

In [30]:
zappos_connection(list(zappos.iterrows())[0])

Failure: 0_0_684832


In [22]:
pool = Pool(processes=multiprocessing.cpu_count())


In [23]:
import time
begin = time.time()
pool.map(zappos_connection, zappos.loc[0:50].iterrows())
print time.time() - begin

0.18887591362


In [24]:
zappos.loc[0:50].iterrows()

<generator object iterrows at 0x10e16c230>